<a href="https://colab.research.google.com/github/Ravi-Teja-konda/AudioInsightsGenerator/blob/main/AudioInsightsGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI-Powered Audio Summarizer, Content Explorer, Perspective Generator, and Storyteller

Welcome to this interactive AI-powered tool that not only summarizes and explores audio content but also adds unique perspectives, generates insightful ideas, and transforms information into engaging stories. This tool, powered by OpenAI's language models, takes the processing of audio content to the next level, allowing you to unlock new avenues of understanding and engagement with your content.

## Features

1. **Audio Summarization**: Upload any audio file and our tool will transcribe it and generate a concise summary. You can choose the format of the summary - bullet points, short paragraphs, or even a single concise sentence.

2. **Filtering Custom Content**: Filter out the noise and focus on what matters to you. Input a keyword or topic of interest, and the tool will sift through the transcript to extract and highlight sections relevant to your input.

3. **Idea Generation and Perspective Building**: This tool helps you look at your content from different angles, generating new ideas and providing unique perspectives that you may have missed.

4.  **Cross-Questioning the Data**: Interact with your content in a more engaging way. Ask follow-up questions related to the filtered content and get relevant answers based on the context of the extracted content.
5. **Storytelling**: Turn dry facts and data into compelling narratives. This storytelling feature takes key points from your audio content and weaves them into engaging narratives, making learning and information retention an enjoyable experience.

## Benefits

**Efficiency**: Save time by extracting key information from lengthy podcasts, lectures, meetings, or any other audio content.

**Personalization**: Engage with the audio content in a personalized and interactive way by filtering content and asking follow-up questions.

**Innovation**: Stimulate your thinking by exploring new perspectives and ideas generated from your content.

**Engagement**: Increase engagement and retention by transforming information into narratives.

**Accessibility**: Make information in audio format more accessible, enabling you to review and engage with it at your own pace and style.

In a nutshell, our AI-powered tool opens up a whole new way of engaging with audio content. It's perfect for students reviewing lecture recordings, professionals catching up on industry podcasts, or any curious individuals eager to extract, explore, and engage with information more efficiently and creatively.


In [ ]:
!pip install openai requests pydub

In [ ]:
import os
import openai
import datetime
import requests
from google.colab import files
from requests.exceptions import RequestException

# You need to set your OpenAI API key here
OPENAI_API_KEY = 'YOUR KEY HERE'
openai.api_key = OPENAI_API_KEY

def download_mp3(url, file_path):
    try:
        # Send a HTTP request to the URL of the MP3 file
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Raise an exception for HTTP errors
        # Open the file in write mode to download the MP3 file
        with open(file_path, 'wb') as audio:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:  # filter out keep-alive new chunks
                    audio.write(chunk)
        print(f"Downloaded successfully: {file_path}")
        return file_path
    except RequestException as e:
        print(f"Error during the request of {url}: {str(e)}")
        return None
    except Exception as e:
        print(f"Error during the download of {url}: {str(e)}")
        return None

# Transcribe audio file using Whisper API
def transcribe_audio_whisper_api(file_path):
    try:
        with open(file_path, "rb") as file:
            transcription = openai.Audio.transcribe("whisper-1", file)
        return transcription
    except Exception as e:
        print(f"{datetime.datetime.now()} [ERROR]: Error in transcription: {e}")
        return None

# Analyze emotion from text using OpenAI GPT model
def analyze_emotion(text):
    try:
        content = f"What emotion is the following text expressing?\n{text}"
        messages = [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": content}]
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k",
            messages=messages,
            max_tokens=200
        )
        emotion = response['choices'][0]['message']['content']
        return emotion.strip()
    except Exception as e:
        print(f"{datetime.datetime.now()} [ERROR]: Error in emotion analysis: {e}")
        return None

# Summarize text using OpenAI Chat model
def summarize_text(text, style):
    try:
        # Using the chat endpoint
        if style == "1":
            content = f"Summarize the following text in bullet points, ordered by importance:\n{text}"
        elif style == "2":
            content = f"Summarize the following text in a short paragraph of 4 to 5 lines:\n{text}"
        else:
            content = f"Summarize the following text concisely:\n{text}"

        messages = [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": content}]
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k",
            messages=messages,
            max_tokens=500
        )
        summary = response['choices'][0]['message']['content']
        return summary
    except Exception as e:
        print(f"{datetime.datetime.now()} [ERROR]: Error in summarization: {e}")
        return None

# User input for source of MP3 file
print("Select the source of MP3 file:")
print("1 - Upload File")
print("2 - Insert MP3 Link")
source = input("Enter 1 or 2: ")

file_path = None
if source == "1":
    # Upload the MP3 file
    uploaded = files.upload()
    for mp3_filename in uploaded.keys():
        print(f"{datetime.datetime.now()} [INFO]: User uploaded file '{mp3_filename}'")
        file_path = mp3_filename
elif source == "2":
    # Download the MP3 file from the given link
    url = input("Enter the MP3 link: ")
    file_path = "audio.mp3"  # you can set to any path you want
    file_path = download_mp3(url, file_path)
else:
    print("Invalid option selected. Please select either 1 or 2.")

if file_path:
    # Transcribe the audio using Whisper API
    print(f"{datetime.datetime.now()} [INFO]: Transcribing audio...")
    transcription = transcribe_audio_whisper_api(file_path)

    # User input for summarization style
    print("Select summarization style:")
    print("1 - Bullet Points")
    print("2 - Short Paragraph")
    print("3 - Concise")
    style = input("Enter 1, 2, or 3: ")

    # Analyze emotion from the transcription using OpenAI GPT model
    if transcription:
        print(f"{datetime.datetime.now()} [INFO]: Analyzing emotion...")
        emotion = analyze_emotion(transcription)
        print(f"{datetime.datetime.now()} [INFO]: Emotion: {emotion}")

        # Summarize the transcription using OpenAI Chat model
        print(f"{datetime.datetime.now()} [INFO]: Summarizing text...")
        summary = summarize_text(transcription, style)
        print(f"{datetime.datetime.now()} [INFO]: Summary: {summary}")

        # Download the summary as a text file
        with open('summary.txt', 'w') as f:
            f.write(f"Emotion: {emotion}\n\n")
            f.write(summary)
        files.download('summary.txt')


Select the source of MP3 file:
1 - Upload File
2 - Download Link
Enter 1 or 2: 2
Enter the MP3 download link: https://dcs.megaphone.fm/ADV8883406591.mp3?key=c4243626894dc56d739288d99305fc87&request_event_id=3ac03551-3a00-4928-ad7c-928000bfe493
Downloaded successfully: audio.mp3
2023-06-29 15:54:56.578581 [INFO]: Transcribing audio...
Select summarization style:
1 - Bullet Points
2 - Short Paragraph
3 - Concise
Enter 1, 2, or 3: 1
2023-06-29 15:55:51.916133 [INFO]: Analyzing emotion...
2023-06-29 15:55:53.151458 [INFO]: Emotion: The emotion expressed in the text can be described as informative and engaging.
2023-06-29 15:55:53.152520 [INFO]: Summarizing text...
2023-06-29 15:56:00.677209 [INFO]: Summary: - Warren Buffett's investing strategy of investing in wonderful companies at a fair price is simple but not commonly followed
- There are many get-rich-quick schemes on the internet, but most of them are scams
- A man named Evaldas Rymouskis successfully scammed Facebook and Google out 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Filtering and Cross-Questioning Custom Content

In this cell, we are implementing a feature that not only allows you to filter custom content from the transcript but also supports cross-questioning the data from the transcript. This can be particularly useful if you are interested in a specific topic or subject matter discussed in the audio file and want to explore it further by asking follow-up questions.

The cell uses OpenAI's GPT model to extract and focus on content that is relevant to a user-specified topic or keyword and generate responses to user-posed questions related to this content.

## How it Works:
1.The user is prompted to input the topic or keyword for which they want to filter the content.

2.The cell then sends a request to OpenAI's GPT model, asking it to extract content that is focused on the topic or keyword specified by the user.

3.The extracted content is displayed, and the user is then able to input a follow-up question related to the extracted content.

4.The cell sends this follow-up question to the GPT model, which generates a response based on the context provided by the extracted content.

5.The user is given the option to download both the filtered content and the question-answer pairs as a text file.

## Use Case:
For instance, if you have a lengthy podcast transcript and you are only interested in the segments that discuss "investment strategies," you can input "investment strategies" as the keyword. The cell will then extract and present the portions of the transcript that are relevant to investment strategies. You can then pose a question like "What are the recommended investment strategies?" and the model will generate a response based on the filtered content.

This feature can save time and effort by helping you to quickly locate and focus on the information that is most relevant to your interests or needs, and deepen your understanding by enabling you to ask follow-up questions.


In [ ]:
# Function to filter custom content from text using OpenAI GPT model
def filter_custom_content(text, user_input):
    try:
        content = f"Extract content from the following text that is focused on {user_input}, and ensure the content in bullet points:\n{text}"
        messages = [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": content}]
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k",
            messages=messages,
            max_tokens=1000
        )
        filtered_content = response['choices'][0]['message']['content']
        return filtered_content.strip()
    except Exception as e:
        print(f"{datetime.datetime.now()} [ERROR]: Error in filtering custom content: {e}")
        return None

# User input for content filtering
user_input = input("Enter the topic you want to filter the content for (e.g. investment strategies): ")

# Filter custom content from the transcript using OpenAI GPT model
if transcription:
    print(f"{datetime.datetime.now()} [INFO]: Filtering custom content...")
    filtered_content = filter_custom_content(transcription, user_input)
    print(f"{datetime.datetime.now()} [INFO]: Filtered Content: {filtered_content}")

    # Download the filtered content as a text file
    with open('filtered_content.txt', 'w') as f:
        f.write(filtered_content)
    files.download('filtered_content.txt')

Enter the topic you want to filter the content for (e.g. investment strategies): what is the hacking method used 
2023-06-29 15:56:34.044064 [INFO]: Filtering custom content...
2023-06-29 15:56:38.505056 [INFO]: Filtered Content: - The hacker used a method known as Business Email Compromise (BEC) scam
- The hacker posed as a company called Quanta Computer and issued a fake invoice to Facebook
- The hacker altered the invoice to change the payment location to their own bank account
- The hacker also conducted a BEC scam against Google using similar techniques
- The hacker set up bank accounts in different countries to launder the stolen money
- The scheme allowed the hacker to extract $23 million from Google and $98 million from Facebook
- The scam was eventually discovered, leading to the hacker's arrest and subsequent sentencing to five years in prison
- BEC scams have resulted in over $10 billion in losses reported to the Internet Crime Complaint Center between 2013 and 2019.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Turning Transcripts into Stories with Real-World Examples

In this cell, we are implementing a feature to turn transcripts into stories integrated with real-world examples. This can make the content more engaging and relatable, especially for educational purposes.

## How it Works:
1. Characters are introduced to represent key concepts from the transcript.
2. A plot is developed aligning with the logical progression of the transcript.
3. The user can choose a narrative style or use the default linear narrative.
4. OpenAI's GPT model is used to create the story.

This approach utilizes storytelling as an effective tool for learning and engagement.


In [ ]:
def turn_transcript_into_story(transcript, narrative_style='linear narrative'):
    try:
        # Prepare content for the chat model
        content = f"Turn the following transcript into an engaging story with real-world examples and ensure the content in bullet points.The narrative style should be a {narrative_style}:\n{transcript}"
        # Creating messages to send to the model
        messages = [{"role": "system", "content": "You are a helpful assistant that can convert transcripts into engaging stories with characters and real-world examples."}, {"role": "user", "content": content}]
        # Sending request to OpenAI API
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k",
            messages=messages,
            max_tokens=1000
        )
        # Extracting the story from the response
        story = response['choices'][0]['message']['content']
        return story.strip()
    except Exception as e:
        print(f"{datetime.datetime.now()} [ERROR]: Error in turning transcript into story: {e}")
        return None

# User input for narrative style (optional)
narrative_style_input = input("Enter the narrative style you want for the story (e.g. nonlinear, hero’s journey) or leave blank for default linear narrative: ")

# Turn transcript into story using OpenAI GPT model
if transcription:
    print(f"{datetime.datetime.now()} [INFO]: Turning transcript into a story...")
    story = turn_transcript_into_story(transcription, narrative_style=narrative_style_input or 'linear narrative')
    print(f"{datetime.datetime.now()} [INFO]: Story: {story}")

    # Download the story as a text file
    with open('story.txt', 'w') as f:
        f.write(story)
    files.download('story.txt')

# Idea Generation and Perspective Building

This cell offers a unique feature of generating ideas and building perspectives from the transcript content. Leveraging the power of OpenAI's GPT model, it extracts unique insights and new ideas, providing a fresh perspective on the discussed content.

## How it Works:
1. The user is asked to input a specific perspective they're interested in. This input is optional, and if left blank, general ideas will be generated.

2. The transcript is then processed through OpenAI's GPT model, which is directed to generate ideas and unique insights from the content.

3. The generated ideas and perspectives are displayed in a bullet-point format. Each unique idea or insight is displayed as a separate bullet point.

4. The user is given the option to download the generated ideas and perspectives as a text file.

## Use cases

This feature can be particularly useful when dealing with informational content where you seek to derive unique insights or brainstorm new ideas. By inputting a specific perspective, you can focus the generated ideas around that theme, thereby gaining targeted insights.

For instance, if you're listening to a business podcast and wish to generate ideas around "innovative marketing strategies", you can input this as your perspective, and the cell will generate ideas centered around this theme.

By offering fresh perspectives and unique insights, this tool can assist in brainstorming sessions, content creation, research, and much more, thus acting as your AI-powered ideation partner

In [ ]:
def generate_perspectives(transcript, perspective=None):
    try:
        # Prepare content for the chat model
        if perspective:
            content = f"Generate unique insights and new ideas from the following transcript with the perspective of {perspective}. Each idea should be a separate bullet point:\n{transcript}"
        else:
            content = f"Generate unique insights and new ideas from the following transcript. Each idea should be a separate bullet point:\n{transcript}"
        # Creating messages to send to the model
        messages = [{"role": "system", "content": "You are a helpful assistant that can generate ideas and build perspectives from a given transcript."}, {"role": "user", "content": content}]
        # Sending request to OpenAI API
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k",
            messages=messages,
            max_tokens=1000
        )
        # Extracting the ideas from the response
        ideas = response['choices'][0]['message']['content']
        return ideas.strip()
    except Exception as e:
        print(f"{datetime.datetime.now()} [ERROR]: Error in generating perspectives: {e}")
        return None

# User input for perspective (optional)
perspective_input = input("Enter a perspective you want for the ideas or leave blank for general ideas: ")

# Generate ideas and perspectives using OpenAI GPT model
if transcription:
    print(f"{datetime.datetime.now()} [INFO]: Generating perspectives from the transcript...")
    ideas = generate_perspectives(transcription, perspective=perspective_input)
    print(f"{datetime.datetime.now()} [INFO]: Generated Ideas and Perspectives: \n{ideas}")

    # Download the ideas as a text file
    with open('ideas.txt', 'w') as f:
        f.write(ideas)
    files.download('ideas.txt')

Enter a perspective you want for the ideas or leave blank for general ideas: BEC as a software
2023-06-29 15:57:13.307844 [INFO]: Generating perspectives from the transcript...
2023-06-29 15:57:19.728009 [INFO]: Generated Ideas and Perspectives: 
- BEC software can detect and prevent phishing attacks by analyzing email headers and domain reputations. 
- BEC software can provide advanced security measures to protect against social engineering attacks, especially in financial transactions.
- BEC software can automate compliance processes for companies, reducing the time spent on manual evidence collection.
- BEC software can integrate with banking systems to identify suspicious transactions and flag potential fraud attempts.
- BEC software can provide training and education materials for employees to increase awareness and prevent falling victim to BEC scams.
- BEC software can monitor and analyze financial activities within a company to detect unusual patterns or suspicious behavior.
- 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>